# NY市場の株価データ解析

daru, statsample, Nyaplot, IRubyを使って、株式市場のブラウン運動を観察する。
収益率の対数が正規分布に従うかどうか。

In [1]:
require "daru"
require "statsample"

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

true

In [2]:
# ニューヨーク市場の銘柄から100のサンプル抽出
# stock_ticker = Daru::DataFrame.from_csv("stock_ticker.csv")
# sample_ticker = stock_ticker.where(stock_ticker["Exchange"].eq("NYQ"))["Ticker"].to_a.shuffle.take(100)
# Daru::DataFrame.new("Ticker" => sample_ticker).write_csv("sample_ticker.csv")

## 株価データのダウンロード

* stock_ticker.csvをもとに、pandasのget_data_yahoo()を使って2006年1月1日以降の株価データを[ダウンロード](StockDownload.ipynb)
* 証券コードに'-'が含まれる4件はエラー

## データのクリーニング

In [ ]:
# pandasでダウンロードしたCSVをロード
# 最初の日付が2006-01-03のものだけ抽出
stocks = Dir["stock/*.csv"].map { |file|
  name = file.slice(/([^\/]+)\.csv\z/, 1)
  Daru::DataFrame.from_csv(file, name: name)
}.select {|stock| stock["Date"][0] == "2006-01-03" }
stocks.size

## 株価の分布

In [ ]:
# 直近1か月の終値の分布
recent_close = Daru::DataFrame.new(
  close: stocks.flat_map { |stock|
    stock["Adj Close"][-30..-1].to_a
  }
)
recent_close.plot type: :histogram, x: :close do |plot, diagram|
  diagram.bin_num(100)
  plot.yrange([0, 200])
  plot.x_label("P($)")
  plot.y_label("No per P")
end

In [ ]:
# 直近1か月の終値の対数の分布
recent_log_close = Daru::DataFrame.new(
  close: stocks.flat_map { |stock|
    stock["Adj Close"].tail(30).map { |x| Math.log(x) }
  }
)
recent_log_close.plot type: :histogram, x: :close do |plot, diagram|
  diagram.bin_num(100)
  plot.yrange([0, 100])
  plot.x_label("ln(P)")
  plot.y_label("No per ln(P)")
end

In [ ]:
# 最初の銘柄の統計量
stocks[0].describe

## 収益率の対数の分布

In [ ]:
# 日次収益率の対数（log(P(t + 1day) / P0(t))）を計算
def get_log_return(stock)
  Daru::DataFrame.new({
    date: stock["Date"].drop(1),
    volume: stock["Volume"].drop(1),
    # [a, b, c, d, ...].each_cons(2) #=> [a, b], [b, c], [c, d], ...
    log_return: stock["Adj Close"].each_cons(2).map { |prev, current| Math.log(current / prev) }
  }, name: stock.name, order: [:date, :volume, :log_return])
end

# 遅い例
def get_log_return_slow(stock)
  Daru::DataFrame.new(
    # 行毎の処理は遅い
    stock.each_row.each_cons(2).map { |prev, current|
      {
        date: current["Date"],
        volume: current["Volume"],
        log_return: Math.log(current["Adj Close"] / prev["Adj Close"])
      }
    }
  )
end

log_returns = stocks.map { |stock|
  get_log_return(stock)
}
nil

In [ ]:
# 最初の3銘柄のヒストグラムをプロット
log_returns.take(3).each do |log_return|
  log_return.plot type: :histogram, x: :log_return do |plot, diagram|
    diagram.bin_num(150)
    plot.yrange([0, 520])
    plot.x_label("ln[P(t + 1d)/P(t)] of #{log_return.name}")
    plot.y_label("No per ln[P(t + 1d)/P(t)]")
  end
end
nil

In [ ]:
# 標本の個々の値から計算した経験累積分布と、標本平均・標本標準偏差から
# 計算した正規累積分布をプロットして比較

def empirical_cumulative_distribution(v)
  x = []
  count = []
  # 簡便さのためsortを使う
  v.sort.each do |i|
    if x.empty? || x[-1] < i
      # xが空またはxの最後の要素がiより小さい時
      x.push(i)
      count.push((count[-1] || 0) + 1) # 最後のcount + 1を追加
    else
      # xの最後の値とiが同じ時
      count[-1] += 1 # 最後のcountをincrement
    end
  end
  # countの値をvの総数で割った値をy (0〜1)とする
  y = count.map { |i| i / v.count.to_f }
  Daru::DataFrame.new(x: x, y: y)
end

def normal_cumulative_distribution(v)
  x = v.min.step(v.max, (v.max - v.min) / 99).to_a
  y = x.map { |i|
    Distribution::Normal.cdf((i - v.mean) / v.std)
  }
  Daru::DataFrame.new(x: x, y: y)
end

def plot_cumulative_distribution(v, x_label)
  ecd = empirical_cumulative_distribution(v)
  ncd = normal_cumulative_distribution(v)
  plot = Nyaplot::Plot.new
  colors = Nyaplot::Colors.qual
  plot.add_with_df(ecd, :line, :x, :y)
  plot.add_with_df(ncd, :line, :x, :y).color("#44bb44")
  plot.xrange([-0.1, 0.1])
  plot.x_label(x_label)
  plot.y_label("Cumulative probability")
  plot.show
end

log_returns.take(3).each do |log_return|
  plot_cumulative_distribution(log_return[:log_return], "ln[P(t + 1d)/P(t)] of #{log_return.name}")
end
nil

In [ ]:
# ダウ平均の累積分布
dji = Daru::DataFrame.from_csv("DJI.csv")
dji_lr = get_log_return(dji)
plot_cumulative_distribution(dji_lr[:log_return], "ln[P(t + 1d)/P(t)] of DJI")

## 標準偏差は取引量の平方根に比例する?

In [ ]:
lr = log_returns[0]
# 対数収益率データに年・月・四半期を追加
df = Daru::DataFrame.new(date: lr[:date], volume: lr[:volume], log_return: lr[:log_return],
  year: lr[:date].map { |d| d.slice(/\A\d{4}/) },
  quarter: lr[:date].map { |d| d.slice(/\A\d{4}-/) + (d.slice(/\A\d{4}-(\d{2})/, 1).to_i / 4 + 1).to_s + "Q" },
  month: lr[:date].map { |d| d.slice(/\A\d{4}-\d{2}/) })

In [ ]:
def get_sqrt_volume_and_std_log_return(df, group)
  x = []
  y = []
  df.group_by(group).each_group { |g|
    x << Math.sqrt(g[:volume].sum)
    y << g[:log_return].std
  }
  Daru::DataFrame.new(sqrt_volume: x, std_log_return: y)
end

last_decade = df.where(df[:year].gteq("2011"))
year = get_sqrt_volume_and_std_log_return(df, :year)
quarter = get_sqrt_volume_and_std_log_return(df, :quarter)
month = get_sqrt_volume_and_std_log_return(df, :month)
plot = Nyaplot::Plot.new
plot.add_with_df(year, :scatter, :sqrt_volume, :std_log_return)
plot.add_with_df(quarter, :scatter, :sqrt_volume, :std_log_return).color("#44bb44")
plot.add_with_df(month, :scatter, :sqrt_volume, :std_log_return).color("#dd8844")
plot.x_label("Square root of volume")
plot.y_label("Standard deviation of ln[P(t + 1d)/P(t)]")
plot.show